Load Data

pip3 install pandas / install pandas

pip3 install numpy / install numpy

pandas documentation: https://pandas.pydata.org/docs/getting_started/intro_tutorials/index.html

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('online_retail.csv', encoding='ISO-8859-1')

print("Dataset Shape:", df.shape)
print("\n" + "="*50)

print("Dataset Info:")
print(df.info()) # Displaying the structure of the DataFrame
print("\n" + "="*50)

print("Statistical Summary:")
print(df.describe(include='all')) # Displaying summary statistics
print("\n" + "="*50)

print("Missing Values:") 
print(df.isnull().sum()) # Displaying the count of missing values
print("\n" + "="*50)

print("Duplicate Rows:", df.duplicated().sum())

Dataset Shape: (1067371, 8)

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB
None

Statistical Summary:
        Invoice StockCode                         Description      Quantity  \
count   1067371   1067371                             1062989  1.067371e+06   
unique    53628      5305                                5698           NaN   
top      537434    85123A  WHITE HANGING HEART T-LIGHT HOLDER           NaN   

In [5]:

# Data Cleaning Steps
df_clean = df.copy()

# Step 1: Remove duplicates
print(f"Original shape: {df_clean.shape}")
df_clean = df_clean.drop_duplicates()
print(f"After removing duplicates: {df_clean.shape}")

# Step 2: Handle cancelled orders (Invoice starting with 'C')
cancelled_orders = df_clean[df_clean['Invoice'].astype(str).str.startswith('C')]
print(f"Cancelled orders found: {len(cancelled_orders)}")
df_clean = df_clean[~df_clean['Invoice'].astype(str).str.startswith('C')]
print(f"After removing cancelled orders: {df_clean.shape}")

# Step 3: Remove records with zero or negative price
print(f"Records with price <= 0: {(df_clean['Price'] <= 0).sum()}")
df_clean = df_clean[df_clean['Price'] > 0]
print(f"After removing zero/negative prices: {df_clean.shape}")

# Step 4: Remove records with zero or negative quantity
print(f"Records with quantity <= 0: {(df_clean['Quantity'] <= 0).sum()}")
df_clean = df_clean[df_clean['Quantity'] > 0]
print(f"After removing zero/negative quantities: {df_clean.shape}")

# Step 5: Handle missing Customer IDs
print(f"Missing Customer IDs: {df_clean['Customer ID'].isnull().sum()}")
# For RFM analysis, we need Customer ID, so we'll create two datasets
df_with_customers = df_clean.dropna(subset=['Customer ID']).copy()
print(f"Dataset with Customer IDs: {df_with_customers.shape}")

# Step 6: Remove non-product stock codes
non_product_codes = ['POST', 'D', 'C2', 'M', 'BANK CHARGES', 'PADS', 'DOT']
df_clean = df_clean[~df_clean['StockCode'].isin(non_product_codes)]
df_with_customers = df_with_customers[~df_with_customers['StockCode'].isin(non_product_codes)]
print(f"After removing non-product codes: {df_clean.shape}")

Original shape: (1067371, 8)
After removing duplicates: (1033036, 8)
Cancelled orders found: 19104
After removing cancelled orders: (1013932, 8)
Records with price <= 0: 6019
After removing zero/negative prices: (1007913, 8)
Records with quantity <= 0: 0
After removing zero/negative quantities: (1007913, 8)
Missing Customer IDs: 228488
Dataset with Customer IDs: (779425, 8)
After removing non-product codes: (1003478, 8)


Data preprocessing completed!
Final dataset shape: (1003478, 16)
Dataset with customers shape: (776624, 16)
